# YouTube 转录

>[YouTube](https://www.youtube.com/) 是由 Google 创建的在线视频分享和社交媒体平台。

本笔记本介绍了如何从 `YouTube transcripts` 加载文档。

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
%pip install --upgrade --quiet  youtube-transcript-api

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
)

In [ ]:
loader.load()

### 添加视频信息

In [ ]:
%pip install --upgrade --quiet  pytube

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=True
)
loader.load()

### 添加语言偏好

`language` 参数：一个语言代码列表，按优先级降序排列，默认为 `en`。

`translation` 参数：一个翻译偏好设置，您可以将可用的字幕翻译成您喜欢的语言。

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg",
    add_video_info=True,
    language=["en", "id"],
    translation="en",
)
loader.load()

### 获取带时间戳的字幕块

获取一个或多个 `Document` 对象，每个对象包含视频字幕的一个块。可以指定块的长度（以秒为单位）。每个块的元数据包含一个 YouTube 视频的 URL，该 URL 将从特定块的开头开始播放视频。

`transcript_format` 参数：`langchain_community.document_loaders.youtube.TranscriptFormat` 的值之一。在此情况下为 `TranscriptFormat.CHUNKS`。

`chunk_size_seconds` 参数：代表每个字幕数据块的视频秒数。默认为 120 秒。

In [ ]:
from langchain_community.document_loaders.youtube import TranscriptFormat

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=TKCMw0utiak",
    add_video_info=True,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30,
)
print("\n\n".join(map(repr, loader.load())))

## YouTube 加载器，来自 Google Cloud

### 先决条件

1. 创建一个 Google Cloud 项目或使用现有项目
1. 启用 [YouTube API](https://console.cloud.google.com/apis/enableflow?apiid=youtube.googleapis.com&project=sixth-grammar-344520)
1. [授权桌面应用的凭据](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)
1. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib youtube-transcript-api`

### 🧑 ingest 您的 Google Docs 数据的说明
默认情况下，`GoogleDriveLoader` 期望 `credentials.json` 文件位于 `~/.credentials/credentials.json`，但这可以通过 `credentials_file` 关键字参数进行配置。`token.json` 同理。请注意，`token.json` 将在您首次使用加载器时自动创建。

`GoogleApiYoutubeLoader` 可以从 Google Docs 文档 ID 列表或文件夹 ID 加载数据。您可以从 URL 获取您的文件夹和文档 ID：
请注意，根据您的设置，可能需要设置 `service_account_path`。更多详情请参见 [此处](https://developers.google.com/drive/api/v3/quickstart/python)。

In [ ]:
# Init the GoogleApiClient
from pathlib import Path

from langchain_community.document_loaders import GoogleApiClient, GoogleApiYoutubeLoader

google_api_client = GoogleApiClient(credentials_path=Path("your_path_creds.json"))


# Use a Channel
youtube_loader_channel = GoogleApiYoutubeLoader(
    google_api_client=google_api_client,
    channel_name="Reducible",
    captions_language="en",
)

# Use Youtube Ids

youtube_loader_ids = GoogleApiYoutubeLoader(
    google_api_client=google_api_client, video_ids=["TrdevFK_am4"], add_video_info=True
)

# returns a list of Documents
youtube_loader_channel.load()